<a href="https://colab.research.google.com/github/GuptaNavdeep1983/CS688/blob/main/Assignment7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#References

http://rstudio-pubs-static.s3.amazonaws.com/256588_57b585da6c054349825cba46685d8464.html

https://cran.r-project.org/web/packages/easyPubMed/vignettes/getting_started_with_easyPubMed.html

http://r-statistics.co/Association-Mining-With-R.html

https://blog.revolutionanalytics.com/2019/02/sequential-pattern-mining-in-r.html

In [ ]:
version

In [ ]:
install.packages("easyPubMed")
install.packages("tm")
install.packages("qdap")

In [ ]:
install.packages("arules")

In [ ]:
install.packages("arulesSequences")

In [ ]:
library(easyPubMed)
library(arules)
library(arulesSequences)
library(dplyr)
years <- c(2020, 2019, 2018, 2017, 2016)
xx <-list()
datalist <- list()
i<-0
for (year in 1:5)
{
  my_query <- paste("((", years[year], "[Date - Create]))", sep = "")
  print(my_query)
  my_entrez_id <- get_pubmed_ids(my_query)
  my_abstracts_xml <- fetch_pubmed_data(my_entrez_id,retstart = 0,  format = "xml", retmax = 100)
  my_PM_list <- articles_to_list(pubmed_data = my_abstracts_xml)
  xx <- lapply(my_PM_list, article_to_df, autofill = TRUE, max_chars = 50)
  full_df <- do.call(rbind, xx)
  datalist[[year]] <- full_df[seq(1, 100, by = 1), c("year", "month", "title", "day")]
  # full_df <- do.call(rbind, xx)
  # print(full_df)
  # df_years[[i]] <- full_df[seq(1, 100, by = 1), c("year", "month", "title")]
}
all_years = do.call(rbind, datalist)

In [135]:
all_years <- unique(all_years)
all_years

,year,month,title,day,month_int,year_int,day_int
,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,2020,11,Assessing the accuracy of conventional gadolinium-enhanced breast MRI in measuring the nodal response to neoadjuvant chemotherapy (NAC) in breast cancer.,11,11,2020,11
2,2020,11,"Camostat Mesylate against SARS-CoV-2 and COVID-19 - Rationale, Dosing and Safety.",11,11,2020,11
3,2020,11,Multicentre Randomised Controlled Trial About the Effect of Individual Reminiscence Therapy in Older Adults with Neurocognitive Disorders.,11,11,2020,11
4,2020,11,Long-term performance analysis of chemical filters in cleanrooms based on a prediction model.,11,11,2020,11
5,2020,11,Dosage of oxytocin for augmentation of labor and women's childbirth experiences: a randomized controlled trial.,11,11,2020,11
6,2020,11,Time Trends in Diabetes Medication Prescription and Factors Associated With Metformin Discontinuation in People With Newly Diagnosed Type 2 Diabetes: a National Population-based Study.,11,11,2020,11
7,2020,11,Ethical Considerations on Pediatric Genetic Testing Results in Electronic Health Records.,11,11,2020,11
8,2020,11,"Transitions from One Electronic Health Record to Another: Challenges, Pitfalls, and Recommendations.",11,11,2020,11
9,2020,11,[Videoassisted Thoracoscopy with Preserved Spontaneous Breathing - an Anaesthesiological Perspective].,11,11,2020,11


In [136]:
#reindex
row.names(all_years) <- NULL


#Sequence Mining

In [137]:
# group articles by year month and day
grouped_years <- all_years %>%
  group_by(year,month, day) %>%
  summarize(
    eventID = as.numeric(paste(year, month, day, sep="")),
    SIZE = n(),
    combined_title = paste(as.character(title), collapse = ' ')
  )
unique_groups = unique(grouped_years)
unique_groups

`summarise()` regrouping output by 'year', 'month', 'day' (override with `.groups` argument)



year,month,day,eventID,SIZE,combined_title
<chr>,<chr>,<chr>,<dbl>,<int>,<chr>
2017,07,07,20170707,1,Drug-dependent functionalization of wild-type and mutant p53 in cisplatin-resistant human ovarian tumor cells.
2017,08,01,20170801,1,Induced cancer stem cells generated by radiochemotherapy and their therapeutic implications.
2017,10,05,20171005,4,"Inhibition of ALDH2 by O-GlcNAcylation contributes to the hyperglycemic exacerbation of myocardial ischemia/reperfusion injury. Gene expression changes in tumor free tongue tissue adjacent to tongue squamous cell carcinoma. Secreted heat shock protein 90 promotes prostate cancer stem cell heterogeneity. Prognostic impact of immune microenvironment in laryngeal and pharyngeal squamous cell carcinoma: Immune cell subtypes, immuno-suppressive pathways and clinicopathologic characteristics."
2017,10,13,20171013,1,Selective Brain Cooling after Traumatic Brain Injury: Effects of Three Different Cooling Methods-Case Report.
2018,02,20,20180220,1,Hypoxia-inducible factor prolyl hydroxylase 2 (PHD2) is a direct regulator of epidermal growth factor receptor (EGFR) signaling in breast cancer.
2018,03,02,20180302,3,"Polyunsaturated fatty acids ameliorate aging via redox-telomere-antioncogene axis. Variation of lifespan in multiple strains, and effects of dietary restriction and BmFoxO on lifespan in silkworm, Bombyx mori. Viral-host interaction in kidney reveals strategies to escape host immunity and persistently shed virus to the urine."
2018,03,13,20180313,1,Activity of the aqueous extract of Schinus terebinthifolius Raddi on strains of the Candida genus.
2018,03,20,20180320,1,Readability of Online Sources Regarding Meniscal Tears.
2018,04,03,20180403,1,Single-Site Robotic Radical Hysterectomy and Sentinel Lymphnode Biopsy in Cervical Cancer: A Case Report.


In [ ]:
# Generate a vector of corpus
library(tm)
text_vec <- VectorSource(unique_groups$combined_title)
vec_corpus <- VCorpus(text_vec)


In [ ]:
# Clean corpus
clean_corpus <- function(corpus){
  corpus <- tm_map(corpus, stripWhitespace)
  corpus <- tm_map(corpus, removePunctuation)
  corpus <- tm_map(corpus, removeNumbers)
  corpus <- tm_map(corpus, content_transformer(tolower))
  corpus <- tm_map(corpus, removeWords, stopwords("en"))
  print(corpus)
    return(corpus)
}
# Apply your customized function to the tweet_corp: clean_corp
clean_corp <- clean_corpus(vec_corpus)
clean_corp

In [ ]:
#Regenerate the data frame from the corpus
dataframe<-data.frame(text=unlist(sapply(clean_corp, `[`, "content")), stringsAsFactors=F)
dataframe$sequenceID<-unique_groups$year
dataframe$eventID<-unique_groups$eventID
dataframe$SIZE<-unique_groups$SIZE
dataframe <- dataframe[order(dataframe$sequenceID, dataframe$eventID),]

# change column order 
dataframe<-dataframe[,c(2,3,1)]
dataframe

In [125]:
# Convert to transaction matrix data type
write.table(dataframe, "mytxtout.txt", sep=" ", row.names = FALSE, col.names = FALSE, quote = FALSE)
trans_matrix <- read_baskets("mytxtout.txt", sep = " ", info = c("sequenceID","eventID"))

inspect(head(trans_matrix))

    items                       sequenceID  eventID
[1] {,                                             
     cells,                                        
     cisplatinresistant,                           
     drugdependent,                                
     functionalization,                            
     human,                                        
     mutant,                                       
     ovarian,                                      
     p,                                            
     tumor,                                        
     wildtype}                        2017 20170707
[2] {,                                             
     cancer,                                       
     cells,                                        
     generated,                                    
     implications,                                 
     induced,                                      
     radiochemotherapy,                            
     stem,  

In [138]:
# Run CSPADE algorithm for sequence analysis
library(arulesSequences)
frequent_pattern <-cspade(trans_matrix,parameter = list(support = 0.5), control = list(verbose = TRUE))


parameter specification:
support : 0.5
maxsize :  10
maxlen  :  10

algorithmic control:
bfstype  : FALSE
verbose  :  TRUE
summary  : FALSE
tidLists : FALSE

preprocessing ... 1 partition(s), 0.01 MB [0.031s]
mining transactions ... 4.43 MB [0.38s]
reading sequences ... [12s]

total elapsed time: 11.942s


In [139]:
as(frequent_pattern, "data.frame")

sequence,support
<chr>,<dbl>
<{}>,1.00
<{activity}>,0.50
<{acute}>,0.50
<{adults}>,0.50
<{analysis}>,0.50
<{associated}>,0.50
<{based}>,0.50
<{brain}>,0.50
<{breast}>,0.50


#Association Mining

In [ ]:

library(qdap)
frequent_terms <- freq_terms(all_years$title, 30)
plot(frequent_terms)

In [128]:
library(tm)
text_vec <- VectorSource(all_years$combined_title)
vec_corpus <- VCorpus(text_vec)


In [129]:
clean_corpus <- function(corpus){
  corpus <- tm_map(corpus, stripWhitespace)
  corpus <- tm_map(corpus, removePunctuation)
  corpus <- tm_map(corpus, removeNumbers)
  corpus <- tm_map(corpus, content_transformer(tolower))
  corpus <- tm_map(corpus, removeWords, stopwords("en"))
  print(corpus)
    return(corpus)
}
# Apply your customized function to the tweet_corp: clean_corp
clean_corp <- clean_corpus(vec_corpus)
clean_corp

<<VCorpus>>
Metadata:  corpus specific: 0, document level (indexed): 0
Content:  documents: 93


<<VCorpus>>
Metadata:  corpus specific: 0, document level (indexed): 0
Content:  documents: 93

In [ ]:
# Create the dtm from the corpus: 
pubmed_dtm <- DocumentTermMatrix(clean_corp)
# Print out tweets_dtm data
pubmed_dtm

In [ ]:
# Since the sparsity is so high, i.e. a proportion of cells with 0s/ cells with other values is too large,
# let's remove some of these low frequency terms
pubmed_dtm_rm_sparse <- removeSparseTerms(pubmed_dtm, 0.98)
# Convert to a matrix: 
pubmed_m <- as.matrix(pubmed_dtm_rm_sparse)
# Print the dimensions
dim(pubmed_m)
# Review a portion of the matrix
pubmed_m[0:10, 0:100]

In [ ]:
library(arules)

rules <- apriori (pubmed_m, parameter = list(supp = 0.001, conf = 0.5))

In [ ]:

rules_conf <- sort (rules, by="confidence", decreasing=TRUE) # 'high-confidence' rules.
rules_conf
inspect(head(rules_conf))

In [ ]:
rules_lift <- sort (rules, by="lift", decreasing=TRUE) # 'high-lift' rules.
inspect(head(rules_lift))

In [ ]:
pubmed.tran<-as(data.frame(dataframe$text),"transactions")
transactionInfo(pubmed.tran)$sequenceID<-unique_groups$year
transactionInfo(pubmed.tran)$eventID<-unique_groups$eventID
transactionInfo(pubmed.tran)$SIZE<-unique_groups$SIZE
inspect(pubmed.tran)